In [1]:
import numpy as np
import pandas as pd

### Preparing the data

To build this model, we will be using the following metdata:
- The genre of the movie.
- The director of the movie. This person is part of the crew.
- The movie's three major stars. They are part of the cast.
- Sub-genres or keywords.

In [2]:
local_data_path = "../data/"
drive_data_path = "/content/drive/MyDrive/Hands-On Recommendation Systems with Python/data/"

In [13]:
df = pd.read_csv(drive_data_path + "metadata_clean.csv")
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995


In [16]:
orig_df = pd.read_csv(drive_data_path + "movies_metadata.csv", low_memory = False)
orig_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [17]:
df['overview'], df['id'] = orig_df["overview"], orig_df["id"]
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [18]:
cred_df = pd.read_csv(drive_data_path + "credits.csv")
display(cred_df.head())

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [19]:
cred_df["cast"][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [20]:
print(cred_df["crew"][0])

[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'depart

In [27]:
key_df = pd.read_csv(drive_data_path + "keywords.csv")
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


### Merge the dataframes

In [24]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [25]:
# convert the ids of df into int
df['id'] = df['id'].apply(clean_ids)

In [26]:
df = df[df['id'].notnull()]

In [28]:
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')

<ipython-input-28-a811f37c5cc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype('int')


In [29]:
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


### Wrangling Keywords, cast and crew

- Convert keywords into a list of strings where each string is a keyword (similar to
genres). We will include only the top three keywords. Therefore, this list can
have a maximum of three elements.
- Convert cast into a list of strings where each string is a star. Like keywords, we
will only include the top three stars in our cast.
- Convert crew into director. In other words, we will extract only the director of
the movie and ignore all other crew members.

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46628 entries, 0 to 46627
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         46624 non-null  object 
 1   genres        46628 non-null  object 
 2   runtime       46360 non-null  float64
 3   vote_average  46624 non-null  float64
 4   vote_count    46624 non-null  float64
 5   year          46628 non-null  int64  
 6   overview      45633 non-null  object 
 7   id            46628 non-null  int64  
 8   cast          46628 non-null  object 
 9   crew          46628 non-null  object 
 10  keywords      46628 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 3.9+ MB


In [31]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
  df[feature] = df[feature].apply(literal_eval)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46628 entries, 0 to 46627
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         46624 non-null  object 
 1   genres        46628 non-null  object 
 2   runtime       46360 non-null  float64
 3   vote_average  46624 non-null  float64
 4   vote_count    46624 non-null  float64
 5   year          46628 non-null  int64  
 6   overview      45633 non-null  object 
 7   id            46628 non-null  int64  
 8   cast          46628 non-null  object 
 9   crew          46628 non-null  object 
 10  keywords      46628 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 3.9+ MB


In [33]:
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [35]:
# get the director only from the crew
def get_director(x):
  for crew_member in x:
    if crew_member['job'] == 'Director':
      return crew_member['name']
  return np.nan

In [37]:
df['director'] = df['crew'].apply(get_director)
df['director'].head()

0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
Name: director, dtype: object

In [41]:
type(df['cast'][0]), type(df['crew'][0])

(list, list)

In [38]:
# return the top 3 elements of for the keywords(sub category/genre) and cast
def generate_list(x):
  if isinstance(x, list):
    names = [element["name"] for element in x]
    if len(names) > 3:
      names = names[:3]
    return names
  return []

In [42]:
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

In [43]:
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [44]:
df[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


The cast "Ryan Reynolds" and "Ryan Gosling" has the same first name and so when we convert them into the vectors, "Ryan" is involved, both recommendation of gosling as well as reynolds are shown.

Hence,

In [45]:
def sanitize(x):
  if isinstance(x, list):
    # strip spaces and convert to lowercase
    return [str.lower(i.replace(" ", "")) for i in x]
  else:
    # check if director exists. If not, return empty string
    if isinstance(x, str):
      return str.lower(x.replace(" ", ""))
    else:
      return ''

In [46]:
for feature in ["cast", "director", "genres", "keywords"]:
  df[feature] = df[feature].apply(sanitize)

### Creating the metadata soup

In [47]:
def create_soup(x):
  return ' '.join(x["keywords"]) + " " + " ".join(x["cast"]) + " " + x["director"] + " " + " ".join(x["genres"])

In [48]:
df["soup"] = df.apply(create_soup, axis=1)

In [49]:
df.iloc[0]['soup']

'jealousy toy boy tomhanks timallen donrickles johnlasseter animation comedy family'

In [55]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director,soup
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgame, disappearance, basedonchildren'sbook]",joejohnston,boardgame disappearance basedonchildren'sbook ...
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, bestfriend, duringcreditsstinger]",howarddeutch,fishing bestfriend duringcreditsstinger walter...
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",forestwhitaker,basedonnovel interracialrelationship singlemot...
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlifecrisis, confidence]",charlesshyer,baby midlifecrisis confidence stevemartin dian...


### Generating the recommendations
- The next steps are identical to the plot description based recommendation system

- Instead of using TF-IDFVectorizer, we will be using CountVectorizer. This is because using
TF-IDFVectorizer will accord less weight to actors and directors who have acted and
directed in a relatively larger number of movies.

- This is not desirable, as we do not want to penalize artists for directing or appearing in
more movies:

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
no_of_movies_to_consider = 1000
count = CountVectorizer(stop_words = "english")
count_matrix = count.fit_transform(df["soup"][:no_of_movies_to_consider])
type(count_matrix), count_matrix.shape

(scipy.sparse._csr.csr_matrix, (1000, 3853))

In [85]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [86]:
type(cosine_sim2), cosine_sim2.shape

(numpy.ndarray, (1000, 1000))

In [77]:
import h5py

dataset_name = 'cosine similarity'
with h5py.File('cosine_sim2.h5', 'w') as hf:
    hf.create_dataset(dataset_name, data=cosine_sim2)

In [65]:
df = df.reset_index()

In [66]:
indices2 = pd.Series(df.index, index=df["title"])

In [69]:
indices2[:3], type(indices2), indices2.shape

(title
 Toy Story           0
 Jumanji             1
 Grumpier Old Men    2
 dtype: int64,
 pandas.core.series.Series,
 (46628,))

In [80]:
# save the pickle file
import pickle
with open('indices.pkl', 'wb') as f:
    pickle.dump(indices2, f)


In [83]:
with open('indices.pkl', 'rb') as f:
    indices2 = pickle.load(f)

indices2[:3], type(indices2), indices2.shape

(title
 Toy Story           0
 Jumanji             1
 Grumpier Old Men    2
 dtype: int64,
 pandas.core.series.Series,
 (46628,))

In [84]:
dataset_name = 'cosine similarity'
with h5py.File('cosine_sim2.h5', 'r') as hf:
    cosine_sim2 = hf[dataset_name][:]

type(cosine_sim2), cosine_sim2.shape

(numpy.ndarray, (1000, 1000))

### Build a Recommender System

In [71]:
def content_recommender(title, cosine_sim=cosine_sim2, df=df, indices=indices2):
  idx = indices[title]
  # sim_scores = [(0, 1.0000000000000002), (1, 0.015706569646092478), (2, 0.0), ...]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for i in sim_scores]
  return df['title'].iloc[movie_indices]

In [72]:
content_recommender('The Lion King')

651                            James and the Giant Peach
770                          The Hunchback of Notre Dame
811                          The Adventures of Pinocchio
986                                             Infinity
66                                              Two Bits
531                               A Simple Twist of Fate
720    Institute Benjamenta, or This Dream People Cal...
7                                           Tom and Huck
39                              Cry, the Beloved Country
216                                             The Cure
Name: title, dtype: object

### Suggestions for improvements

- Experiment with number of keywords, genres and cast
- Come up with more well-defined sub-genres
- Give more weight to the director
- Consider other members of the crew
- Experiment with other metadata
- Introduce a popularity filter